# A script that generates urls for iids from the ESGF API (
removing this from the runtime recipe generation should speed up submission by a lot!

- Only overwrite when forced? Later

In [11]:
# from transforms import StoreToZarrLegacyDynamic as StoreToZarr
import json
from pyesgf.search import SearchConnection

iids = []
with open('tim_iids.txt') as f:
    for line in f:
        if not '#' in line:
            
            # iids.append(str(line).replace(" ", "").replace("',\n", "").replace("'",""))
            iids.append(line.strip().replace('\n', ''))
list(iids)

# try the MIROC ones that were not available again
iids = []

In [12]:
# iids = {}

In [13]:
## Query ESGF for the urls
iid_schema = 'mip_era.activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.version'

conn = SearchConnection(
    "https://esgf-node.llnl.gov/esg-search",
    distrib=True
)
recipe_input_dict = {}
for iid in iids:
    try:
        context_kwargs = {'replica':None,'facets':['doi']} # this assumes that I can use replicas just as master records. I think that is fine
        for label, value in zip(iid_schema.split('.'), iid.split('.')):
            context_kwargs[label] = value

        # is this a problem with the `v...` in version?
        context_kwargs['version'] = context_kwargs['version'].replace('v','')

        # testing
        # del context_kwargs['version']
        ctx = conn.new_context(**context_kwargs)
        print(f"{iid}: Found {ctx.hit_count} hits")

        results = ctx.search() # these might include several data nodes (curiously even if I set replica to false?)

        if len(results)<1:
            print(f'Nothing found for {iid}. Skipping.')
        else:
            # for now take the first one that has any urls at all
            for ri,result in enumerate(results):
                print(f"{iid}-{result.dataset_id}: Extracting URLS")
                urls = [f.download_url for f in result.file_context().search()] # this raises an annoying warning each time. FIXME
                if len(urls)>0:
                    recipe_input_dict[iid] = {}
                    recipe_input_dict[iid]['urls'] = urls
                    # populate this to pass to the database later
                    recipe_input_dict[iid]['instance_id'],recipe_input_dict[iid]['data_node'] = result.dataset_id.split('|')
                    break
    except:
        print('failed')

CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r10i1p1f1.day.sfcWind.gn.v20200623: Found 2 hits
CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r10i1p1f1.day.sfcWind.gn.v20200623-CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r10i1p1f1.day.sfcWind.gn.v20200623|esgf-data2.diasjp.net: Extracting URLS



-------------------------------------------------------------------------------
Warning - defaulting to search with facets=*

This behavior is kept for backward-compatibility, but ESGF indexes might not
successfully perform a distributed search when this option is used, so some
results may be missing.  For full results, it is recommended to pass a list of
facets of interest when instantiating a context object.  For example,

      ctx = conn.new_context(facets='project,experiment_id')

Only the facets that you specify will be present in the facets_counts dictionary.

This warning is displayed when a distributed search is performed while using the
facets=* default, a maximum of once per context object.  To suppress this warning,
set the environment variable ESGF_PYCLIENT_NO_FACETS_STAR_WARNING to any value
or explicitly use  conn.new_context(facets='*')

-------------------------------------------------------------------------------


CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r10i1p1f1.day.sfcWind.gn.v20200623-CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r10i1p1f1.day.sfcWind.gn.v20200623|esgf-data1.llnl.gov: Extracting URLS



-------------------------------------------------------------------------------
Warning - defaulting to search with facets=*

This behavior is kept for backward-compatibility, but ESGF indexes might not
successfully perform a distributed search when this option is used, so some
results may be missing.  For full results, it is recommended to pass a list of
facets of interest when instantiating a context object.  For example,

      ctx = conn.new_context(facets='project,experiment_id')

Only the facets that you specify will be present in the facets_counts dictionary.

This warning is displayed when a distributed search is performed while using the
facets=* default, a maximum of once per context object.  To suppress this warning,
set the environment variable ESGF_PYCLIENT_NO_FACETS_STAR_WARNING to any value
or explicitly use  conn.new_context(facets='*')

-------------------------------------------------------------------------------


CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r11i1p1f1.day.sfcWind.gn.v20200623: Found 2 hits
CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r11i1p1f1.day.sfcWind.gn.v20200623-CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r11i1p1f1.day.sfcWind.gn.v20200623|esgf-data1.llnl.gov: Extracting URLS


...


-------------------------------------------------------------------------------
Warning - defaulting to search with facets=*

This behavior is kept for backward-compatibility, but ESGF indexes might not
successfully perform a distributed search when this option is used, so some
results may be missing.  For full results, it is recommended to pass a list of
facets of interest when instantiating a context object.  For example,

      ctx = conn.new_context(facets='project,experiment_id')

Only the facets that you specify will be present in the facets_counts dictionary.

This warning is displayed when a distributed search is performed while using the
facets=* default, a maximum of once per context object.  To suppress this warning,
set the environment variable ESGF_PYCLIENT_NO_FACETS_STAR_WARNING to any value
or explicitly use  conn.new_context(facets='*')

-------------------------------------------------------------------------------


CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r9i1p1f1.day.sfcWind.gn.v20200623: Found 2 hits
CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r9i1p1f1.day.sfcWind.gn.v20200623-CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r9i1p1f1.day.sfcWind.gn.v20200623|esgf-data2.diasjp.net: Extracting URLS



-------------------------------------------------------------------------------
Warning - defaulting to search with facets=*

This behavior is kept for backward-compatibility, but ESGF indexes might not
successfully perform a distributed search when this option is used, so some
results may be missing.  For full results, it is recommended to pass a list of
facets of interest when instantiating a context object.  For example,

      ctx = conn.new_context(facets='project,experiment_id')

Only the facets that you specify will be present in the facets_counts dictionary.

This warning is displayed when a distributed search is performed while using the
facets=* default, a maximum of once per context object.  To suppress this warning,
set the environment variable ESGF_PYCLIENT_NO_FACETS_STAR_WARNING to any value
or explicitly use  conn.new_context(facets='*')

-------------------------------------------------------------------------------


CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r9i1p1f1.day.sfcWind.gn.v20200623-CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r9i1p1f1.day.sfcWind.gn.v20200623|esgf-data1.llnl.gov: Extracting URLS



-------------------------------------------------------------------------------
Warning - defaulting to search with facets=*

This behavior is kept for backward-compatibility, but ESGF indexes might not
successfully perform a distributed search when this option is used, so some
results may be missing.  For full results, it is recommended to pass a list of
facets of interest when instantiating a context object.  For example,

      ctx = conn.new_context(facets='project,experiment_id')

Only the facets that you specify will be present in the facets_counts dictionary.

This warning is displayed when a distributed search is performed while using the
facets=* default, a maximum of once per context object.  To suppress this warning,
set the environment variable ESGF_PYCLIENT_NO_FACETS_STAR_WARNING to any value
or explicitly use  conn.new_context(facets='*')

-------------------------------------------------------------------------------


In [14]:
# result.json

In [15]:
import gcsfs
import json
with open('/Users/juliusbusecke/Downloads/keys/leap-pangeo-dataflow_service_account.json') as f:
    token = json.load(f)
fs = gcsfs.GCSFileSystem(token=token) # auth here...

for k,v in recipe_input_dict.items():
    with open(f"{k}.json", 'w') as file:
        json.dump(v, file)
    fs.put(f"{k}.json", f"gs://leap-persistent/jbusecke/cmip6urls/{k}.json")

In [16]:
set(iids) - set(recipe_input_dict.keys())

{'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r10i1p1f1.day.sfcWind.gn.v20200623',
 'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r11i1p1f1.day.sfcWind.gn.v20200623',
 'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r12i1p1f1.day.sfcWind.gn.v20200623',
 'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r13i1p1f1.day.sfcWind.gn.v20200623',
 'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r14i1p1f1.day.sfcWind.gn.v20200623',
 'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r15i1p1f1.day.sfcWind.gn.v20200623',
 'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r16i1p1f1.day.sfcWind.gn.v20200623',
 'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r17i1p1f1.day.sfcWind.gn.v20200623',
 'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r18i1p1f1.day.sfcWind.gn.v20200623',
 'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r19i1p1f1.day.sfcWind.gn.v20200623',
 'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r20i1p1f1.day.sfcWind.gn.v20200623',
 'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r4i1p1f1.day.sfcWind.gn.v20200623',
 'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r5i1p1f1.day.sfcWind.gn.v20200623',
 

In [17]:
# list(recipe_input_dict.keys())